In [ ]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 csv 使用的工具
import csv

# 執行 command 的時候用的
import os

# 引入 hashlib 模組
import hashlib

# 引入 regular expression 工具
import re

from bs4 import BeautifulSoup

'''
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
'''
# 啟動瀏覽器工具的選項
options = webdriver.ChromeOptions()
prefs = {
    'profile.default_content_setting_values':
        {
            'notifications': 2
        }
}
# options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
options.add_argument("--start-maximized")         #最大化視窗
#options.add_argument("--incognito")               #開啟無痕模式
options.add_argument("--disable-popup-blocking ") #禁用彈出攔截
options.add_argument("disable-infobars")  # 關閉'chrome正受到自動測試軟件的控制'提示

# 使用 Chrome 的 WebDriver (含 options)
driver = webdriver.Chrome( options = options )

#視窗大小設定 (寬，高)
#driver.set_window_size(1200, 960)

# driver.maximize_window() #視窗最大化
# driver.minimize_window() #視窗最小化


'''
自訂變數
'''
# 放置單一店家資訊
family_info = []
# 放置所有店家資訊
family_infos = []

#全域變數
page = 1
index = 0

def visit():
    
    global page
    
    # 前往指定連結
    driver.get('https://findbiz.nat.gov.tw/fts/query/QueryBar/queryInit.do');
    sleep(3)
    #輸入名稱
    txtInput = driver.find_element(By.CSS_SELECTOR, "#qryCond")
    txtInput.send_keys("23060248")
    #按下送出
    sleep(2)
    btnInput = driver.find_element(By.CSS_SELECTOR, "#qryBtn")
    btnInput.click()
    sleep(3.5)
    #點選詳細資料
    info = driver.find_element(By.CSS_SELECTOR, "#vParagraph > div > div:nth-child(2) > span.moreLinkMouseOut")
    info.click()
    sleep(3)
    #點選分公司資料
    branch_btn = driver.find_element(By.CSS_SELECTOR,"#tabBrCmpy")
    branch_btn.click()
    sleep(2)
    driver.find_element_by_xpath('//*[@id="QueryCmpyDetail_queryCmpyDetail"]/nav/ul/li[1]/a').click()
    #print("有點第一頁")
    check_situation()
    print("---------爬完第"+str(page)+"頁---------")
    page+=1
    
    
# 爬取2-10頁的店家資料
def next_page_2_10():
    
    global page

    for num in range(4,14):
        
        
        page_xpath = '//*[@id="QueryCmpyDetail_queryCmpyDetail"]/nav/ul/li[' + str(num) + ']/a'
#         print('num:'+str(num))
#         print('page_xpath'+page_xpath)
        sleep(3)
        driver.find_element_by_xpath(page_xpath).click()
        check_situation()
        
        print("---------爬完第"+str(page)+"頁---------")
        page+=1
        
        if num == 13 :
            next_page_11_249()

# 爬取11-249頁的店家資料
def next_page_11_249():
    
    global page
    
    for ten_num in range(24):
        for num in range(5,15):
            
            #print('n:'+ str(n))
            page_xpath = '//*[@id="QueryCmpyDetail_queryCmpyDetail"]/nav/ul/li[' + str(num) + ']/a'
#             print('ten_num:'+str(ten_num))
#             print('num:'+str(num))
#             print('page_xpath'+page_xpath)
            sleep(3)
            driver.find_element_by_xpath(page_xpath).click()
        
            check_situation()
            print("---------爬完第"+str(page)+"頁---------")
            page+=1

    
def check_situation():
    
    global index 
    
    while 1 :
        try:
            for num in range(0,20):
                sleep(0.5)

                index_xpath = '//*[@id="tabBrCmpyContent"]/div/table[1]/tbody/tr[' + str(num+1) +']/td[1]'
                index = driver.find_element_by_xpath(index_xpath).text

                situation_xpath = '//*[@id="tabBrCmpyContent"]/div/table[1]/tbody/tr[' + str(num+1) +']/td[4]'
                situation_text = driver.find_element_by_xpath(situation_xpath).text

                name_xpath = '//*[@id="tabBrCmpyContent"]/div/table[1]/tbody/tr[' + str(num+1) +']/td[3]/a'
                sleep(2.5)
                
                print("第"+str(page)+"頁_"+str(index)+"_"+driver.find_element_by_xpath(name_xpath).text)
                
                driver.find_element_by_xpath(name_xpath).click()
                #print("有點進分店資料")
                if situation_text == '廢止':
                    get_data_repeal()
                else:
                    get_data()


                #print(driver.find_element_by_xpath(name_xpath).text)
            break
        except:
            if len(driver.find_element_by_xpath("/html/body/table/tbody/tr/td/table/tbody/tr/td/span/a")) > 0:
                print("按太快")
                driver.find_element_by_xpath("/html/body/table/tbody/tr/td/table/tbody/tr/td/span/a").click()
                continue
            else:
                print("查看分店列表有問題,Continue")
                driver.execute_script("location.reload()")
                driver.find_element_by_xpath('//*[@id="bs-example-navbar-collapse-1"]/ul/li[3]/a').click()
                continue
            
    
def get_data_repeal():
    
    global index
    
    family_info = ['','','','','','','','','','',''] 
    sleep(0.25)

    while 1 :
        
        try:
            for num in range(0,11):
                sleep(1)
                info_xpath = '//*[@id="tabCmpyContent"]/div/table/tbody/tr[' + str(num+1) + ']/td[2]'
                #print(driver.find_element_by_xpath(info_xpath).text)

                if num == 0:
                    sleep(0.1)
                    family_info[num] = driver.find_element_by_xpath(info_xpath).text[:-5]
                elif num == 2:
                    sleep(0.1)
                    addr_len = driver.find_element_by_xpath(info_xpath).text.find('司')
                    family_info[num] = driver.find_element_by_xpath(info_xpath).text[:addr_len+1]
                elif num == 4:
                    sleep(0.1)
                    family_info[num] = driver.find_element_by_xpath(info_xpath).text[:-7]
                elif num == 9:
                    sleep(0.1)
                    family_info[num] = '第'+str(page)+'頁'
                elif num == 10:
                    sleep(0.1)
                    family_info[num] = index
                else:
                    sleep(0.1)
                    family_info[num] = driver.find_element_by_xpath(info_xpath).text
#             print(family_info)
            family_infos.append(family_info)
            
            
            sleep(1.5)
            driver.find_element_by_xpath('//*[@id="bs-example-navbar-collapse-1"]/ul/li[3]/a').click()
    
            break
            
        except:
            print("進分店資訊按太快")
            driver.find_element_by_xpath("/html/body/table/tbody/tr/td/table/tbody/tr/td/span/a").click()
            continue
        
def get_data():
    
    global index
    
    family_info = ['','','','','','','','','','',''] 
    sleep(0.25)

    while 1 :
        
        try:
            for num in range(0,11):
                info_xpath = '//*[@id="tabCmpyContent"]/div/table/tbody/tr[' + str(num+1) + ']/td[2]'
#                 print(driver.find_element_by_xpath(info_xpath).text)

                if num == 0:
                    sleep(0.1)
                    family_info[num] = driver.find_element_by_xpath(info_xpath).text[:-5]
                    print(family_info[num])
                elif num == 2:
                    sleep(0.1)
                    addr_len = driver.find_element_by_xpath(info_xpath).text.find('司')
                    family_info[num] = driver.find_element_by_xpath(info_xpath).text[:addr_len+1]
                    print(family_info[num])
                elif num == 4:
                    sleep(0.1)
                    family_info[num] = driver.find_element_by_xpath(info_xpath).text[:-7]
                    print(family_info[num])
                elif num == 7:
                    family_info[num+1] = driver.find_element_by_xpath(info_xpath).text
                    print(family_info[num])
                elif num == 8:
                    continue
                elif num == 9:
                    sleep(0.1)
                    family_info[num] = '第'+str(page)+'頁'

                elif num == 10:
                    sleep(0.1)
                    family_info[num] = index

                else:
                    sleep(0.1)
                    family_info[num] = driver.find_element_by_xpath(info_xpath).text

#             print(family_info)
            family_infos.append(family_info)
            
            
            sleep(2)
            driver.find_element_by_xpath('//*[@id="bs-example-navbar-collapse-1"]/ul/li[3]/a').click()
    
            break
            
        except:
            print("進分店資訊按太快")
            driver.find_element_by_xpath("/html/body/table/tbody/tr/td/table/tbody/tr/td/span/a").click()
            continue
        
visit()
next_page_2_10()


In [ ]:
def create_csv():
    family_infos.insert(0,['分公司統一編號','分公司狀況','分公司名稱','分公司經理姓名','分公司所在地','登記機關','核准設立日期','廢止日期','最後核准變更日期','頁碼','序號'])
    #print(family_infos)
    csvfile=open('family_infos.csv','w', newline='',encoding="utf-8")
    obj = csv.writer(csvfile)
    for row in family_infos:
        obj.writerow(row)
    csvfile.close()    
    
create_csv()